# Relational Database Structure for LTD

In [1]:
from datascience import *
import re
import numpy as np
import os

# Seperates Speaker/Speech pairs from file

In [2]:
def sep_speech(string):
    parse_file = ''
    with open(string) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('\n', '')
    parse_file = parse_file.replace('Mr. President', 'MrPresident')
    
    split = re.split(r'Mr. |Ms. |Mrs. ', parse_file)
    split.pop(0)
    name_and_speech = make_array()
    for i in np.arange(len(split)):
        try:
            lastname = re.match('\w*\. ', split[i]).group(0)[:-2]
            name_and_speech = np.append(name_and_speech, lastname)
            value = re.sub('[A-Z]\w*\. ', '', split[i])
            name_and_speech = np.append(name_and_speech, value)
        except:
            abcabcabc = 1
    return name_and_speech

## Speech Table

In [3]:
speeches = Table().with_columns("speech_id", make_array(), 
                                "speaker_id", make_array(), 
                                "proceeding_id", make_array(), 
                                "topic_id", make_array(), 
                                "word_count", make_array(), 
                                "speech_text", make_array(),
                                'file_name', make_array())

In [4]:
count = 0
for file in os.listdir("/Users/cun-yuwang/Desktop/Congress_Records"): #change directory once we have everything
    if file.endswith(".txt"):
        separated = sep_speech(file)
        i = 0
        while i < len(separated):
            row = make_array()
            text = separated[i+1]
            if len(text) > 30:
                row = [count, separated[i], 'proceeding_id', 'topic-id', len(text.split()), text, file] 
                count += 1
                speeches = speeches.with_row(row)     
            i +=2


In [5]:
speeches

speech_id,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name
0,CORNYN,proceeding_id,topic-id,2309,"MrPresident, many recall that Christmas came a little ea ...",CREC-2018-01-03-pt2-PgS10.txt
1,DURBIN,proceeding_id,topic-id,18,"MrPresident, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS10.txt
2,DURBIN,proceeding_id,topic-id,893,"MrPresident, on September 5 of last year, Attorney Gener ...",CREC-2018-01-03-pt2-PgS11.txt
3,WHITEHOUSE,proceeding_id,topic-id,23,"MrPresident, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS11.txt
4,WHITEHOUSE,proceeding_id,topic-id,2035,"Thank you, MrPresident, and happy new year to you. For ...",CREC-2018-01-03-pt2-PgS13.txt
5,INHOFE,proceeding_id,topic-id,22,"MrPresident, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS13.txt
6,INHOFE,proceeding_id,topic-id,25,"MrPresident, I ask unanimous consent to speak in morning ...",CREC-2018-01-03-pt2-PgS13.txt
7,INHOFE,proceeding_id,topic-id,1673,"MrPresident, I begin by wishing everyone a happy new yea ...",CREC-2018-01-03-pt2-PgS13.txt
8,INHOFE,proceeding_id,topic-id,1682,"MrPresident, I mentioned that there were three things I ...",CREC-2018-01-03-pt2-PgS16-2.txt
9,INHOFE,proceeding_id,topic-id,10,I yield the floor. The PRESIDING The Senator from ...,CREC-2018-01-03-pt2-PgS16-2.txt


## Speaker Table

In [6]:
speakers = Table().with_columns("speaker_id", make_array(), 
                                "first_name", make_array(), 
                                "last_name", make_array(), 
                                "type" , make_array(),
                                "party", make_array(), 
                                "state", make_array(), 
                                "district", make_array(),
                                "bio_guide_id", make_array(),
                                "congress_id", make_array())

In [7]:
speakers

speaker_id,first_name,last_name,type,party,state,district,bio_guide_id,congress_id


## Topic Table

In [8]:
topics = Table().with_columns("topic_id", make_array(), 
                                "topic_name", make_array())

In [9]:
topics

topic_id,topic_name


## Proceedings Table

In [10]:
proceedings = Table().with_columns("proceeding_id", make_array(), 
                              "year", make_array(),
                                   'month', make_array(),
                                   'day', make_array(),
                              "title", make_array(),
                                  'File Name', make_array())

In [11]:
def sep_date_from_file(file):
    abcdef = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', file)
    return re.split('-', abcdef[0])
sep_date_from_file('CREC-2018-01-04-pt1-PgS47.xml')[0]

'2018'

In [12]:
def find_title(string):
    parse_file = ''
    with open(string) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('Mr. President', 'MrPresident')
    title = re.findall('[A-Z \'-]+[A-Z0-9-,\. ]*[Continued]*\\n', parse_file)
    return title[0].strip()

In [14]:
count = 0
for file in speeches.group('file_name').column(0):
    if file.endswith(".txt"):
        date = sep_date_from_file(file)
        try:
            title = find_title(file)
        except:
            title = 'No Title'
        row = [count, date[0], date[1], date[2], title, file]
        proceedings = proceedings.with_row(row)
        count += 1

        
    

In [104]:
proceedings.show()

proceeding_id,year,month,day,title,File Name
0,2018,01,03,TAX REFORM BILL AND WORK BEFORE THE SENATE,CREC-2018-01-03-pt2-PgS10.txt
1,2018,01,03,DACA,CREC-2018-01-03-pt2-PgS11.txt
2,2018,01,03,CLIMATE CHANGE,CREC-2018-01-03-pt2-PgS13.txt
3,2018,01,03,TAX REFORM BILL AND GOVERNMENT REGULATION,CREC-2018-01-03-pt2-PgS16-2.txt
4,2018,01,03,IRAN,CREC-2018-01-03-pt2-PgS16.txt
5,2018,01,03,NOMINATION OF JOHN ROOD,CREC-2018-01-03-pt2-PgS18.txt
6,2018,01,03,EXECUTIVE SESSION,CREC-2018-01-03-pt2-PgS19-3.txt
7,2018,01,03,HONORING DOUGLAS COUNTY SHERIFF'S DEPUTY ZACKARI PARRISH,CREC-2018-01-03-pt2-PgS19.txt
8,2018,01,03,WELCOMING NEW SENATORS,CREC-2018-01-03-pt2-PgS2-2.txt
9,2018,01,03,FUNDING THE GOVERNMENT,CREC-2018-01-03-pt2-PgS2-3.txt
